<a href="https://colab.research.google.com/github/pedrogengo/mlops-course-activities/blob/main/homework_week_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet

--2022-05-23 02:58:21--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.224.65
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.224.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-01.parquet’

fhv_tripdata_2021-0 100%[===================>]  11.33M  52.1MB/s    in 0.2s    

2022-05-23 02:58:21 (52.1 MB/s) - ‘fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]



In [3]:
import pandas as pd

In [4]:
df = pd.read_parquet('/content/fhv_tripdata_2021-01.parquet')
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


## Q1

How many records are on the file?

In [5]:
len(df)

1154112

## Q2

What's the average trip duration in January?

In [6]:
df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
df['duration'] = df['duration'].apply(lambda x: x.total_seconds() / 60)

In [7]:
df['duration'].mean()

19.1672240937939

## Data preparation

In [8]:
df = df[(df['duration'] <= 60) & (df['duration'] >= 1)]

## Q3

What's the fractions of missing values for the pickup location ID?

In [9]:
df['PUlocationID'].isna().sum() / len(df)

0.8352732770722617

In [10]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)

In [11]:
df_model = df[['PUlocationID', 'DOlocationID', 'duration']].copy()

## Q4
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

In [12]:
from sklearn.feature_extraction import DictVectorizer

In [13]:
df_model[['PUlocationID', 'DOlocationID']] = df_model[['PUlocationID', 'DOlocationID']].astype(str)

In [14]:
train_dicts = df_model[['PUlocationID', 'DOlocationID']].to_dict(orient='records')

In [15]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [16]:
len(dv.feature_names_)

525

## Q5

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [17]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [18]:
target = df['duration']

In [19]:
rg = LinearRegression()
rg.fit(X_train, target)

LinearRegression()

In [20]:
np.sqrt(mean_squared_error(target, rg.predict(X_train)))

10.528519107210744

## Q6

Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

In [21]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet

--2022-05-23 03:22:27--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.174.73
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.174.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10645466 (10M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-02.parquet’

fhv_tripdata_2021-0 100%[===================>]  10.15M  48.8MB/s    in 0.2s    

2022-05-23 03:22:27 (48.8 MB/s) - ‘fhv_tripdata_2021-02.parquet’ saved [10645466/10645466]



In [30]:
df_test = pd.read_parquet('/content/fhv_tripdata_2021-02.parquet')
df_test.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


In [31]:
df_test['duration'] = df_test['dropOff_datetime'] - df_test['pickup_datetime']
df_test['duration'] = df_test['duration'].apply(lambda x: x.total_seconds() / 60)

In [32]:
df_test = df_test[(df_test['duration'] <= 60) & (df_test['duration'] >= 1)]

In [33]:
df_test['PUlocationID'] = df_test['PUlocationID'].fillna(-1)
df_test['DOlocationID'] = df_test['DOlocationID'].fillna(-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
df_test[['PUlocationID', 'DOlocationID']] = df_test[['PUlocationID', 'DOlocationID']].astype(str)

In [35]:
df_test_clean = df_test[['PUlocationID', 'DOlocationID', 'duration']].copy()

In [36]:
test_dicts = df_test_clean[['PUlocationID', 'DOlocationID']].to_dict(orient='records')

In [37]:
X_test = dv.transform(test_dicts)

In [38]:
np.sqrt(mean_squared_error(df_test_clean['duration'], rg.predict(X_test)))

11.014283196111764